# Scoring Roller Elements
*Components, calculations, analysis of the elements that make up the paper scoring mechanism*

## Overview
The paper scoring mechanism consists of the following main components:
* pair of rollers that fit tightly together to indent the paper with the folding pattern
* stepper motor to drive the rollers
* supporting pulleys, bearings, belt, and shafts

## Summary
Stepper Motor with encoder - [Encoder Motor Link](https://www.omc-stepperonline.com/p-series-nema-17-closed-loop-stepper-motor-48ncm-67-99oz-in-with-encoder-1000cpr.html?search=P%20Series%20Nema%2017%20Closed%20Loop%20Stepper%20Motor%2048Ncm%2F67.99oz.in%20with%20Encoder%201000CPR)

Belt - [Belt Link](http://shop.sdp-si.com/catalog/?cid=p342&filter=a0%3A2%3ADouble%20Sided%3Ba1%3A2%3A3%20mm%20(GT2)%3Ba4%3A2%3ANeoprene%3Ba15%3A2%3AFiberglass&sort=undefined&view=table)

Pulleys - [Pulley Link](http://shop.sdp-si.com/catalog/?cid=p218&filter=a0%3A2%3A3%20mm%20(GT2)%3Ba2%3A2%3APolycarbonate&sort=undefined&view=table)

## Design Methodology
1. Since a design has already been choosen, model the rollers
2. Determine the force neccesary to score the paper
3. Determine the torque, speed, and gearing requirements for the motor
4. Model the components, determine the layout, create assembly
5. Purchase materials
6. Build mechanism
7. Test mechanism

### Model the Rollers
The folding pattern has already been determined. It consists of a mirui-ori pattern in which each facet is a parallelogram the same size as two equalateral triangles joined together. They are sized in a way that 6 facets fill the 8.5" direction on the paper.

<img src="images/onesheet.png" width=400><img src="images/roller.png" width=400>

### Scoring Force
Using a partially 3d printed roller the force was estimated to be 10 lb and that a soft backing material is needed. Roughly extrapolating this out across a full length roller it is estimated to require 30 lb of force. 30A, 50A, and 70A rated rubber was purchased to be used as a backing material. The rubber is in sheet 1/16" with an adhesive backing.

[Silicone 50A](https://www.grainger.com/product/E-JAMES-Rubber-Sheet-1MXF9)<img src="https://static.grainger.com/rp/s/is/image/Grainger/1MWV2_AS01?hei=536&wid=536&$adapimg$=" width=250>
[Buna-N 70A](https://www.grainger.com/product/E-JAMES-Rubber-Sheet-1TYD1?searchQuery=1TYD1&searchBar=true)<img src="https://static.grainger.com/rp/s/is/image/Grainger/1DTW4_AS01?hei=536&wid=536&$adapimg$=" width=250>
[Neoprene 30A](https://www.grainger.com/product/E-JAMES-Rubber-Sheet-1DXC5?searchQuery=1DXC5&searchBar=true)<img src="https://static.grainger.com/rp/s/is/image/Grainger/1DTW4_AS01?hei=536&wid=536&$adapimg$=" width=250>

In [1]:
#Load - W= 30 lb
load= 30

## Forces on Motor
The torque/speed required by the motor is dependent on a few assumption:

1. rolling friction force
2. desired speed
3. moment of inertia of the rollers
4. angular acceleration
5. gearing ratio

### Rolling Friction Force
[Rolling Friction](https://www.mhi.org/media/members/14220/130101690137732025.pdf)
<img src="images/rfwheel.png" width=250><img src="images/rfequation.png" width=250><img src="images/rfmaterial.png" width=250><img src="images/radius.png" width=250>


In [2]:
load # (lbf)
frictioncoeff= .06 # (in) assumed to be slightly higher than polyurethane
radius=1.44 # (in) from the solid model
rollingforce= frictioncoeff*(load/radius)
rollingforce #lbf
rollingtorque=rollingforce*radius #lb-in
print("The torque required to overcome rolling friction is",rollingtorque,"lb-in")

The torque required to overcome rolling friction is 1.7999999999999998 lb-in


### Speed
To allow room for improving the operational speed of the device, the scoring roller should roughly be capable of moving a sheet through in under 1 sec.

In [3]:
import math as math

diameter=2*radius
revpersheet=11.5/(diameter*math.pi)
rollerrpm=revpersheet*60
print("The roller speed is",rollerrpm,"rpm")

The roller speed is 76.26174356486652 rpm


### Moment of Inertia
The MOI of the roller can be determined from Solidworks if the proper material is applied and the density adjusted for the infill pattern.
For simplification we will assume a solid cylinder and the actual mass used by the 3d printer to create the roller.
The slicer program, Cura, shows the mass to be 306 grams.

I=1/2 *m * r^2 for solid cylinders


How much torque is needed to accerate the rollers from 0 rev/s to 3 rev/s in .5s?

Will need the moment of inertia of any roller or shaft and multiply by 2 since there is 2

In [4]:
# I=1/2 *m * r^2 for solid cylinders, which we will assume now foor simplification
massrollerkg= .306  # kg
massrollerlb= massrollerkg*2.205

radiusrollerin= radius

momentrollerlbin2=.5*massrollerlb*radiusrollerin**2
print("The roller's moment of inertia is",momentrollerlbin2,"lb-in^2")

The roller's moment of inertia is 0.699560064 lb-in^2


The shaft will also have a moment of inertia.The shaft used is [Shaft Link](https://www.mcmaster.com/catalog/126/1219) 8632T1

A D shaped profile is suitable for driving the rollers, as a set screw of keyed method would experience excess stress in the printed PLA plastic.

<img src="https://www.mcmaster.com/mvA/Contents/gfx/ImageCache/863/8632t21-@1x_636861859180528217.png?ver=ImageNotFound,/mvA/Contents/gfx/ImageCache/863/8632t21-@2x_636861859180528217.png?ver=ImageNotFound,/mvA/Contents/gfx/ImageCache/863/8632t21-@3x_636861859180528217.png?ver=ImageNotFound,/mvA/Contents/gfx/ImageCache/863/8632t21-@4x_636861859180528217.png?ver=ImageNotFound" width=300>

In [5]:
densityshaftkgperm3= 8050 # kg/m3  steel
shaftdiameterIN=3/8
shaftdiameterM=shaftdiameterIN/39.37
shaftlengthIN=11
shaftlengthM=shaftlengthIN/39.37
shaftvolumeM3=shaftlengthM*math.pi*(shaftdiameterM/2)**2
massshaftkg= densityshaftkgperm3*shaftvolumeM3
massshaftlb= 2.205*massshaftkg

shaftradiusIN= shaftdiameterIN/2

momentshaftlbin2=.5*massshaftlb*shaftradiusIN**2
print("The shaft's moment of inertia is",momentshaftlbin2,"lb-in^2")

The shaft's moment of inertia is 0.006211920374729777 lb-in^2


### Angular Acceleration
We will assume that we need the device to get to the desired speed within 1s

$\alpha$=$\delta$$\omega$/$\delta$$t$

In [6]:
initialomega=0 #rps
finalomega=rollerrpm/60 #rps
deltat=1 #s
alpharps2= (finalomega-initialomega)/deltat
alpha=alpharps2*2*math.pi

print("The angular acceleration is",alpha,"rad/s^2")

The angular acceleration is 7.986111111111112 rad/s^2


### Torque Required at the Roller
This will be the sum of the torques from the angular acceleration of the shaft and roller, plus the torque to overcome the rolling friction.
There are 2 shafts and rollers so this needs to be taken into account.

$T$=$I$$x$$\alpha$

In [7]:
#torque required equals the sum of the torques with the acceleration described above
# T(N*m)=I(kg*m^2)*a(rad/s^2)
# Tam=Itotal*(3 rps*(2pi rad/1 rev))*(1/.5s)
# Ttotal=Trf+Tam

torqueroller=momentrollerlbin2*alpha
torqueshaft=momentshaftlbin2*alpha
rollingtorque

torquetotal=2*(torqueroller+torqueshaft)+rollingtorque

print("The total amount of torque required at the rollers in total is",torquetotal,"lb-in")


The total amount of torque required at the rollers in total is 13.072746972651935 lb-in


### Motor Selection
Ideally would like to use a NEMA 17 sized motor to drive the scoring rollers. 
[Motor Link](https://www.omc-stepperonline.com/nema-17-bipolar-59ncm-84oz-in-2a-42x48mm-4-wires-w-1m-cable-and-connector.html)

<img src="https://www.omc-stepperonline.com/image/cache/catalog/stepper-motor/17/nema-17-bipolar-59ncm-84ozin-2a-42x48mm-4wires-w-1m-cable-and-connector-110-1000x1000.jpg" width=300>

In [8]:
# START Setting up Iframe and PDF class to pull in pdf
import IPython.display as iframe
class PDF(object):
  def __init__(self, pdf, size=(200,200)):
    self.pdf = pdf
    self.size = size

  def _repr_html_(self):
    return '<iframe src={0} width={1[0]} height={1[1]}></iframe>'.format(self.pdf, self.size)

  def _repr_latex_(self):
    return r'\includegraphics[width=1.0\textwidth]{{{0}}}'.format(self.pdf)
# END

PDF('https://www.omc-stepperonline.com/download/17HS19-2004S1.pdf',size=(800,550))

In [9]:
PDF('https://www.omc-stepperonline.com/download/17HS19-2004S1_Torque_Curve.pdf',size=(800,550))

For positional control a motor with a built in encoder may prove useful.

Due to prices and lead times from multiple sources the closest option that has an encoder is [Encoder Motor Link](https://www.omc-stepperonline.com/p-series-nema-17-closed-loop-stepper-motor-48ncm-67-99oz-in-with-encoder-1000cpr.html?search=P%20Series%20Nema%2017%20Closed%20Loop%20Stepper%20Motor%2048Ncm%2F67.99oz.in%20with%20Encoder%201000CPR)

<img src="https://www.omc-stepperonline.com/image/cache/catalog/closed-loop/water%20proof%20with%20brake/17E1K-05%2001-1000x1000.jpg" width=300>

In [10]:
PDF("https://www.omc-stepperonline.com/download/17E1K-05.pdf", size=(800,550))

In [11]:
PDF("https://www.omc-stepperonline.com/download/17E1K-05_Torque_Curve.pdf", size=(800,550))

Motor selection will be dependent on the gearing ratio. Which is also dependent on what size of pulleys can fit in the package spaces.

Pulleys and belt considerations are:
- GT2
- GT3
- And their corresponding belts

[Belt Link](http://shop.sdp-si.com/catalog/?cid=p342&filter=a0%3A2%3ADouble%20Sided%3Ba1%3A2%3A3%20mm%20(GT2)%3Ba4%3A2%3ANeoprene%3Ba15%3A2%3AFiberglass&sort=undefined&view=table)

[Pulley Link](http://shop.sdp-si.com/catalog/?cid=p218&filter=a0%3A2%3A3%20mm%20(GT2)%3Ba2%3A2%3APolycarbonate&sort=undefined&view=table)

Alternatively the pulleys can be 3d printed. Multiple configurations have been modeled in SolidWorks for use in the project.

<img src="images/GT3pulley.png" width=400>


Below is a great source for timing belts and pulleys:

In [12]:
PDF("https://www.sdp-si.com/PDFS/Technical-Section-Timing.pdf", size=(550,800))

In [13]:
PDF("https://www.sdp-si.com/ss/pdf/80502151.pdf", size=(550,800))

**The motor is able to run much faster than needed** *see torque curves*

So we will use the highest gear ratio where the pulleys fit within the package.

The smallest GT2/GT3 pulley, driving pulley, is 16 teeth.

The largest that fits as the driven pulleys is 72 teeth or less.

<img src="images/pulleylayout.png" width=400>

An idler pulley is used to prevent interference and will also have a tensioning device to prevent slack in the belt.

This layout follows guidance for a minimum wrap angle of 60 degrees and a minimum tooth engagment of 6 teeth.


In [14]:
# Gear ratio

drivingteeth=16
driventeeth=72
gearratio=driventeeth/drivingteeth
print("The max gear ratio is",gearratio)

The max gear ratio is 4.5


In [15]:
# Max needed speed of the motor at given gear ratio

motorrpm=rollerrpm*gearratio
print("The max needed motor speed is",motorrpm,"rpm")


The max needed motor speed is 343.17784604189933 rpm


It is estimated from the torque curve that at this speed the torque is .4 N*m

In [16]:
motortorque=.4*8.851 #converting to lb*in
print(motortorque,"lb-in")

3.5404000000000004 lb-in


At the roller this tanslates to:

In [17]:
torqueatrollers=motortorque*gearratio
print(torqueatrollers,"lb-in")

15.931800000000003 lb-in


In [18]:
print("Does the torque output of",round(torqueatrollers,2),"lb-in","meet the requirement of",round(torquetotal,2),"lb-in ?","  ",torqueatrollers>torquetotal)

Does the torque output of 15.93 lb-in meet the requirement of 13.07 lb-in ?    True
